In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

In [2]:
stemmer = LancasterStemmer

In [4]:
import numpy 
import numpy as np
import tflearn
import tensorflow as tf
import random
import json

ModuleNotFoundError: No module named 'tflearn'

In [5]:
with open("intents.json") as file:
     data = json.load(file)

NameError: name 'json' is not defined

In [6]:
words = []
labels = []
docs_x = []
docs_y = []

In [7]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

NameError: name 'data' is not defined

In [8]:
     
    if intent["tag"] not in labels:
        labels.append(intent["tag"])

NameError: name 'intent' is not defined

In [9]:
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))

In [10]:
labels = sorted(labels)

In [11]:
training = []
output = []


In [15]:
out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    
    wrds = [stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)
    
training = numpy.array(training)
output = np.array(output)


In [17]:
import tensorflow as tf
tf.reset_default_graph()

SyntaxError: invalid syntax (Temp/ipykernel_11896/3615827415.py, line 1)

In [ ]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

In [ ]:
model = tflearn.DNN(net)

model.fit(training, output, n_epoch=2000, batch_size=8, show_metric=True)
model.save("model.tflearn")